In [12]:
import numpy as np
import gym
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras.optimizers import Adam
import keras.backend as K
import math
import time
import matplotlib.pyplot as plt
import tensorflow as tf

In [13]:
learning_rate=0.01
done_reward = 0
n_max_episodes = 1000
n_max_steps = 10000
gamma = 0.95
epsilon = 0.0001

In [14]:
env = gym.make("CartPole-v0")
n_action_space = env.action_space.n

c:\users\zhchxiao\anaconda3\envs\pytorch\lib\site-packages\gym\envs\registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [23]:
def lossFunction(y_true, y_pred):
    return - K.log(y_pred[0]) * y_pred[1] - y_true

In [24]:
def buildModel():
    input = Input(shape=[4])
    x = Dense(32, activation='relu')(input)
    x = Dense(16, activation='relu')(x)
    output = Dense(n_action_space, activation='softmax')(x)
    
    model = Model(inputs=input, outputs=output)
    model.compile(loss=lossFunction, optimizer=Adam(lr=learning_rate))
    return model


def discountReward(rewards):
    discount_rewards = np.zeros_like(rewards, dtype=np.float32)
    cumulative = 0.
    for i in reversed(range(len(rewards))):
        cumulative = cumulative * gamma + rewards[i]
        discount_rewards[i] = cumulative
        
    discount_rewards_mean = np.mean(discount_rewards)
    discount_rewards_std = np.std(discount_rewards)
    discount_rewards = (discount_rewards - discount_rewards_mean) / discount_rewards_std
    return discount_rewards


model = buildModel()

In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 4)                 0         
_________________________________________________________________
dense_15 (Dense)             (None, 32)                160       
_________________________________________________________________
dense_16 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_17 (Dense)             (None, 2)                 34        
Total params: 722
Trainable params: 722
Non-trainable params: 0
_________________________________________________________________


In [ ]:
episode_rewards = []
average_episode_rewards = []
total_step = []
start_time = time.time()

for episode in range(n_max_episodes):
    rewards = []
    states = []
    actions = []
    state = env.reset()
    state = state.reshape(-1. 4)
    for step in range(n_max_steps):
        prob_action = model.predict(state)
        action = np.random.choice(np.array(range(n_action_space)), size=1, p=prob_action.revel())
        next_state, reward, done, _ = env.step(action[0])
        
        next_state = next_state.reshape(-1, 4)
        states.append(state)
        rewards.append(reward)
        actions.append(prob_action)
        state = next_state
        if done:
            break
    episode_reward = sum(rewards)
    episode_rewards.append(episode_reward)
    states = np.array(states).reshape(-1, 4)
    discount_rewards = discountReward(rewards).reshape(-1, 1)
    actions = np.array(actions), reshape(-1, 1)
    
    X = [state, actions, discount_rewards]